In [2]:
EGO_ROOT = '/home/zhangyl/videollm-online/datasets/ego4d/'
CAPTION_ROOT = '/home/zhangyl/videollm-online/datasets/ego4d_action_caption'
MOVE_CAPTION_ROOT = '/home/zhangyl/videollm-online/datasets/ego4d_move_action_caption'

import json
import os
import tqdm

from video_caption_action_scene import AnnotationLoader, BetaAlphaCalculator

EGO_VERSION_ROOT = os.path.join(EGO_ROOT, 'v2')
json_path = os.path.join(EGO_ROOT, 'ego4d.json')
train_path = f'{EGO_VERSION_ROOT}/annotations/refined_narration_stream_train.json'
val_path = f'{EGO_VERSION_ROOT}/annotations/refined_narration_stream_val.json'
origin_path = f'{EGO_VERSION_ROOT}/annotations/all_narrations_redacted.json'
video_root = f'{EGO_VERSION_ROOT}/full_scale_2fps'
alpha = 4.9
device = 'cuda:3'
caption_dir = '/root/videollm-online/tmp5'

annotation_loader = AnnotationLoader(train_path, val_path, origin_path, json_path)
data = annotation_loader.get_data()
origin_narration = annotation_loader.get_origin_narration()

beta_alpha_calculator = BetaAlphaCalculator(data, alpha)
beta_alpha_calculator.compute_beta()
beta_map = beta_alpha_calculator.get_beta_map()
alpha = beta_alpha_calculator.get_alpha()

train_caption = json.load(open(f'{CAPTION_ROOT}/action_caption_train.json'))
val_caption = json.load(open(f'{CAPTION_ROOT}/action_caption_val.json'))
all_caption = {**train_caption, **val_caption}

move_train_caption = json.load(open(f'{MOVE_CAPTION_ROOT}/action_caption_train.json'))
move_val_caption = json.load(open(f'{MOVE_CAPTION_ROOT}/action_caption_val.json'))
move_all_caption = {**move_train_caption, **move_val_caption}
video2scene = json.load(open('/home/zhangyl/videollm-online/data/estp/ego4d/metafile/video2scene.json'))

video2scene = json.load(open('/home/zhangyl/videollm-online/data/estp/ego4d/metafile/video2scene.json'))

/home/zhangyl/miniconda3/envs/videollm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(json.dumps(all_caption['9970b072-f426-4f60-95bd-b922f458a3de'],indent=4))

{
    "fe576856-1ad4-4377-95f8-7f9ecebcf35d": [
        {
            "caption": "With a swift, deliberate motion, you lift your right hand from the wooden floor, showcasing its muscular definition and natural skin tone. The action is fluid, highlighting the strength and dexterity of your fingers as they curl slightly before returning to rest on the surface beside the saw machine.",
            "stamp_time": 540.0,
            "start_time": 539.4812753372536,
            "end_time": 540.5187246627464,
            "simi": 0.9337199926376343,
            "action_narration": "You lift your hand.",
            "action_idx": 0,
            "text": "With a swift, deliberate motion, you lift your right hand from the wooden floor, showcasing its muscular definition and natural skin tone. The action is fluid, highlighting the strength and dexterity of your fingers as they curl slightly before returning to rest on the surface beside the saw machine.",
            "time": 540.5187246627464
      

In [10]:
def merge_caption(captions, move_caption,video_uid, clip_uid, video2scene, origin_narration, w_pre = False):
    caption = captions[video_uid][clip_uid]
    move_caption = move_caption[video_uid][clip_uid]
    
    caption_texts = ""
    all_caption = caption + move_caption
    all_caption = sorted(all_caption, key = lambda x: x['start_time'])
    
    for action_idx, cap in enumerate(all_caption):
        if cap in caption:
            action_narration = 'Time is {}-{}. Action narration is \"'.format(cap['start_time'], cap['end_time']) + cap['action_narration'] + '\".\n'
            caption_text = 'Detailed Description: \"' + cap['text'] + '\" \n'
            caption_text = action_narration + caption_text
            caption_texts += caption_text + '\n'
        elif cap in move_caption:
            if 'stamp_time' not in cap and w_pre:
                action_narration = 'Time is {} - {}.\n'.format(cap['start_time'], cap['end_time'])
            elif 'stamp_time' not in cap and not w_pre:
                continue
            else:
                action_narration = 'Time is {}.\n'.format(cap['stamp_time'])
            caption_text = 'Detailed Description: \"' + cap['text'] + '\".\n'
            caption_text = action_narration + caption_text
            caption_texts += caption_text + '\n\n'
        else:
            raise ValueError('Error')
                
    return caption_texts

def caption_merger(captions,move_captions, video2scene, origin_narration, w_pre = False):
    for video_uid in move_captions.keys():
        for clip_uid in move_captions[video_uid].keys():
            caption_texts = merge_caption(captions,move_captions, video_uid, clip_uid, video2scene, origin_narration, w_pre = False)
            yield caption_texts, video_uid, clip_uid

merger = caption_merger(all_caption, move_all_caption, video2scene, origin_narration, w_pre = True)

In [26]:
print(video_uid, clip_uid)

1cc9a153-cec3-4ec5-8e85-78c775653bd2 a003760a-fb37-4cbc-9ca8-6db79d3b0029


In [11]:
text, video_uid, clip_uid = next(merger)
print(text)

Time is -0.8127154879941165-0.8301632213274499. Action narration is "You hold.".
Detailed Description: "As you hold the transparent, cylindrical object with a metallic nozzle in your right hand, you notice that it is resting on a white towel spread out on the table. The towel has some yellow stains and appears to be slightly crumpled. In the background, various items are scattered around, including black cables, a small round mirror, a white container with a red label, and a few other indistinct objects. On the left side of the frame, there's another person wearing a dark-colored hoodie with white text or design on it, but only their hands and arms are visible as they work with something off-camera." 

Time is 2.2395945120058838-3.88247322132745. Action narration is "You pick up the towel.".
Detailed Description: "With your right hand, you pick up a yellow towel from the table. The towel appears slightly damp and has some greenish stains on it, indicating recent use. You hold the towel

In [17]:
def print_json(caption_texts, video_uid, clip_uid):
    print(json.dumps({'video_uid': video_uid, 'clip_uid': clip_uid, 'caption': caption_texts}, indent=4))

In [27]:
def merge_caption(captions, move_caption,video_uid, clip_uid, video2scene, origin_narration, w_pre = False):
    caption = captions[video_uid][clip_uid]
    move_caption = move_caption[video_uid][clip_uid]
    # print_json(move_caption, video_uid, clip_uid)
    caption_texts = ""
    all_caption = caption + move_caption
    all_caption = sorted(all_caption, key = lambda x: x['start_time'])
    
    for action_idx, cap in enumerate(all_caption):
        if cap in caption:
            action_narration = 'Time is {}-{}. Action narration is \"'.format(cap['start_time'], cap['end_time']) + cap['action_narration'] + '\".\n'
            # caption_text = 'Event Detailed Description: \"' + cap['text'] + '\" \n'
            caption_text = ''
            caption_text = action_narration + caption_text
            caption_texts += caption_text + '\n'
        elif cap in move_caption:
            action_narration = 'Time is {} - {}.\n'.format(cap['start_time'], cap['end_time'])
            caption_text = 'Surrounding Detailed Description: \"' + cap['text'] + '\".\n'
            caption_text = action_narration + caption_text
            caption_texts += caption_text + '\n'
        else:
            raise ValueError('Error')
                
    return caption_texts



context_text = merge_caption(all_caption, move_all_caption, video_uid, clip_uid, video2scene, origin_narration, w_pre = True)
print(context_text, video_uid, clip_uid)

Time is -0.8127154879941165-0.8301632213274499. Action narration is "You hold.".

Time is 0.0210286 - 274.48828273153157.
Surrounding Detailed Description: "The video showcases a first-person perspective of an individual engaged in a mechanical or automotive task within a workshop environment. The primary focus is on the person's hands, which are clad in black gloves with red accents and occasionally hold various tools and parts.

Initially, the individual is seen holding a large cylindrical object, likely a part of a vehicle such as a headlight or similar component, against a backdrop of a cluttered workbench covered with a white cloth stained with grease or paint. Tools and materials are scattered around, indicating an active workspace.

As the video progresses, the individual manipulates the cylindrical object, cleaning it with a yellow cloth. The movement is methodical, suggesting careful handling to ensure thorough cleaning without damaging the surface. The camera remains steady t

In [3]:
a = [{1:3,3:5}]
b = [{1:4,3:6}]

c =a + b

for d in c:
    if d in a:
        print(a)
    if d in b:
        print(b) 


[{1: 3, 3: 5}]
[{1: 4, 3: 6}]
